In [17]:
import statsmodels.stats.power as smp
import math
import scipy
import numpy as np
import pandas

In [27]:
# Set globals

alpha = 0.05 # the false positive rate
power = 0.80 # 1-false negative rate
sigma = 10  # standard deviation
mde = 0.10
mu_a = 100
mu_b = mu_a*(1+mde)
d = (mu_b-mu_a)/sigma

n_obs = math.ceil(smp.tt_solve_power(effect_size=d, nobs=None, alpha=alpha, power=power, alternative='two-sided'))
n_obs

10

In [21]:
# set the "true" effect to 0
effect = 0
mu_b = (1+effect)*mu_a
n_mc = 100

v_a = sigma * np.random.randn(n_obs,n_mc) + mu_a
v_b = sigma * np.random.randn(n_obs,n_mc) + mu_b

ttr = scipy.stats.ttest_ind(v_a, v_b)
ttr.pvalue < alpha

# confirm the FPR is as expected
sum(ttr.pvalue < alpha)/len(ttr.pvalue)

0.05

In [22]:
from IPython.display import display, Math, Latex
display(Math(r'F(k) = \int_{-\infty}^{\infty} f(x) e^{2\pi i k} dx'))

<IPython.core.display.Math object>

In [26]:
%%latex
This is some text
\begin{aligned}
\nabla \times \vec{\mathbf{B}} -\, \frac1c\, \frac{\partial\vec{\mathbf{E}}}{\partial t} & = \frac{4\pi}{c}\vec{\mathbf{j}} \\
\nabla \cdot \vec{\mathbf{E}} & = 4 \pi \rho \\
\end{aligned}

<IPython.core.display.Latex object>